In [1]:
import spotipy 
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials


I'm going to focus my song recommender on the top best albums of all time as listed by my trusted website to discover alternative music, Rate Your Music. Luckily, I found this dataset on Kaggle containing the top 5000 albums.

In [2]:
top_albums = pd.read_csv(r"C:\Users\angel\Desktop\Data Analysis\Ironhack\Song Recommendation Project\Data\rym_clean1.csv")
top_albums.head(3)

,Unnamed: 0,position,release_name,artist_name,release_date,release_type,primary_genres,secondary_genres,descriptors,avg_rating,rating_count,review_count
0,1,1,OK Computer,Radiohead,1997-06-16,album,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ...",4.24,74027,1541
1,2,2,Kid A,Radiohead,2000-10-03,album,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp...",4.23,61658,751
2,3,3,The Dark Side of the Moon,Pink Floyd,1973-03-23,album,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi...",4.21,60192,1557


Here I split the album names into two batches to avoid saturating the rate limit of the Spotify API. I also corrected a few album titles that I recognized were misspelled

In [14]:
top_albums["release_name"] = top_albums["release_name"].replace({"Lift Yr. Skinny Fists Like Antennas to Heaven!":"Lift Your Skinny Fists Like Antennas to Heaven",
                     "★ [Blackstar]":"Blackstar",
                     "Black Sabbath Vol. 4":"Vol. 4",
                     "Kyuss [Welcome to Sky Valley]": "Welcome To Sky Valley",
                     "Trans Europa Express":"Trans-Europe Express",
                     "무너지기 (Crumbling)":"Crumbling",
                     "Red House Painters [Rollercoaster]":"Red House Painters I"})

top_500 = top_albums[top_albums["position"] <= 500]
second_top_500 = top_albums[(top_albums["position"] > 500) & (top_albums["position"] <= 1000)]

second_top_500_names = second_top_500["release_name"]

Here I define de Spotipy token

In [4]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= "af3a4e21d9974f798b0ddef081728f2b", client_secret= "99a65d20eff04d64bcf24b11824dffc4"))
results = sp.search(q=f'album: Kid Amnesiac', limit=1, type="album")
results

{'albums': {'href': 'https://api.spotify.com/v1/search?query=album%3A+Kid+Amnesiac&type=album&offset=0&limit=1',
  'items': [{'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4Z8W4fKeB5YxbusRsdQVPb'},
      'href': 'https://api.spotify.com/v1/artists/4Z8W4fKeB5YxbusRsdQVPb',
      'id': '4Z8W4fKeB5YxbusRsdQVPb',
      'name': 'Radiohead',
      'type': 'artist',
      'uri': 'spotify:artist:4Z8W4fKeB5YxbusRsdQVPb'}],
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     'NL',
     'NZ',
     'NI',
     'NO',
     'PA',
     'PY',
     'PE',
     'PH',
     'PL',
     'PT',
     'SG',

This function, get_audio_features, takes a list of album names as input. It searches for each album on Spotify, retrieves its tracks, and then extracts audio features for each track using the Spotify API. If audio features are found for all tracks, it creates a DataFrame containing the audio features along with the album and song names. Finally, it returns a concatenated DataFrame of all albums' audio features. If no audio features are found for any album, it returns None. Additionally, it prints out messages indicating the progress and any issues encountered during the process.

In [5]:
dfs = []

def get_audio_features(albums):
    for album in albums:
        results = sp.search(q=f'album:{album}', limit=1, type="album")
        if len(results['albums']['items']) == 0:
            print(f"No se encontró ningún álbum con el nombre '{album}'.")
            continue
        album_id = results['albums']['items'][0]['id']
        tracks = sp.album_tracks(album_id)
        track_ids = [track['id'] for track in tracks['items']]
        song_names = [track['name'] for track in tracks['items']]
        audio_features = sp.audio_features(track_ids)
        if all(audio_features):
            df = pd.DataFrame(audio_features)
            df['album'] = album
            df['song name'] = song_names
            dfs.append(df)
            print(f"{album}")
        else:
            print(f"No se encontraron caracteristicas de audio para las canciones de {album}")

    if len(dfs) == 0:
        print("No se encontraron datos para ningún álbum.")
        return None
    
    return pd.concat(dfs, ignore_index=True)

In [6]:
get_audio_features(top_500_names)

OK Computer
Kid A
The Dark Side of the Moon
Loveless
My Beautiful Dark Twisted Fantasy
In Rainbows
Wish You Were Here
In the Aeroplane Over the Sea
The Bends
To Pimp a Butterfly
The Velvet Underground & Nico
In the Court of the Crimson King
Abbey Road
Sgt. Pepper's Lonely Hearts Club Band
Nevermind
Revolver
good kid, m.A.A.d city
The Rise and Fall of Ziggy Stardust and the Spiders From Mars
Unknown Pleasures
Yeezus
Amnesiac
The Money Store
Madvillainy
Funeral
The Wall
Remain in Light
Animals
The College Dropout
Pet Sounds
Paranoid
The Beatles [White Album]
The Queen Is Dead
Lift Your Skinny Fists Like Antennas to Heaven
Led Zeppelin [IV]
Igor
Doolittle
Hail to the Thief
The Life of Pablo
Rubber Soul
Spiderland
In utero
Late Registration
Is This It
Graduation
Illmatic
Blonde
Damn.
Flower Boy
The Doors
Master of Puppets
KIDS SEE GHOSTS
Dummy
Kind of Blue
Pink Moon
A Moon Shaped Pool
Disintegration
808s & Heartbreak
Weezer [Blue Album]
Souvlaki
Low
ye
Closer
Selected Ambient Works 85-92
L

My Favorite Things
The Fragile
Leviathan
NEU!
Freak Out!
To See the Next Part of the Dream
xx
Reflektor
Piece of Mind
†
Thick as a Brick
Bright Green Field
The Final Cut
LP!
2112
Elliott Smith
Damaged
Bridge Over Troubled Water


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album,song name
0,0.305,0.872,2,-7.313,1,0.0503,0.007860,0.800000,0.2610,0.648,167.959,audio_features,7c378mlmubSu7NGkLFa4sN,spotify:track:7c378mlmubSu7NGkLFa4sN,https://api.spotify.com/v1/tracks/7c378mlmubSu...,https://api.spotify.com/v1/audio-analysis/7c37...,287880,4,OK Computer,Airbag
1,0.251,0.848,5,-6.501,1,0.0579,0.037700,0.005910,0.0545,0.207,163.709,audio_features,6LgJvl0Xdtc73RJ1mmpotq,spotify:track:6LgJvl0Xdtc73RJ1mmpotq,https://api.spotify.com/v1/tracks/6LgJvl0Xdtc7...,https://api.spotify.com/v1/audio-analysis/6LgJ...,387213,4,OK Computer,Paranoid Android
2,0.312,0.595,0,-8.919,1,0.0298,0.048400,0.001390,0.1030,0.337,150.502,audio_features,2CVV8PtUYYsux8XOzWkCP0,spotify:track:2CVV8PtUYYsux8XOzWkCP0,https://api.spotify.com/v1/tracks/2CVV8PtUYYsu...,https://api.spotify.com/v1/audio-analysis/2CVV...,267693,3,OK Computer,Subterranean Homesick Alien
3,0.293,0.276,4,-11.357,0,0.0349,0.224000,0.122000,0.1670,0.195,121.603,audio_features,0z1o5L7HJx562xZSATcIpY,spotify:track:0z1o5L7HJx562xZSATcIpY,https://api.spotify.com/v1/tracks/0z1o5L7HJx56...,https://api.spotify.com/v1/audio-analysis/0z1o...,267187,4,OK Computer,Exit Music (For A Film)
4,0.351,0.676,9,-9.017,1,0.0313,0.000121,0.113000,0.1800,0.143,102.460,audio_features,2fuYa3Lx06QQJAm0MjztKr,spotify:track:2fuYa3Lx06QQJAm0MjztKr,https://api.spotify.com/v1/tracks/2fuYa3Lx06QQ...,https://api.spotify.com/v1/audio-analysis/2fuY...,299560,4,OK Computer,Let Down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,0.445,0.680,8,-11.338,1,0.0528,0.378000,0.000209,0.1020,0.750,163.664,audio_features,5Bh8l8evdBSIoaK6EP1bWI,spotify:track:5Bh8l8evdBSIoaK6EP1bWI,https://api.spotify.com/v1/tracks/5Bh8l8evdBSI...,https://api.spotify.com/v1/audio-analysis/5Bh8...,195760,4,Bridge Over Troubled Water,Baby Driver
5800,0.411,0.381,11,-12.361,1,0.0286,0.130000,0.257000,0.1320,0.524,76.963,audio_features,5MbXzXGbqobR8xPVPs8OXA,spotify:track:5MbXzXGbqobR8xPVPs8OXA,https://api.spotify.com/v1/tracks/5MbXzXGbqobR...,https://api.spotify.com/v1/audio-analysis/5MbX...,238413,4,Bridge Over Troubled Water,The Only Living Boy in New York
5801,0.745,0.771,8,-10.067,1,0.0352,0.355000,0.000315,0.0791,0.932,117.209,audio_features,0nNI8eyMjDGjPDuQq87aJb,spotify:track:0nNI8eyMjDGjPDuQq87aJb,https://api.spotify.com/v1/tracks/0nNI8eyMjDGj...,https://api.spotify.com/v1/audio-analysis/0nNI...,166173,4,Bridge Over Troubled Water,Why Don't You Write Me
5802,0.431,0.963,6,-7.823,1,0.0677,0.177000,0.000071,0.8330,0.637,152.527,audio_features,1tk4uuDWxk0DTx2X5rGSxe,spotify:track:1tk4uuDWxk0DTx2X5rGSxe,https://api.spotify.com/v1/tracks/1tk4uuDWxk0D...,https://api.spotify.com/v1/audio-analysis/1tk4...,172773,4,Bridge Over Troubled Water,"Bye Bye Love - Live at Memorial Auditorium, Bu..."


In [12]:
data = pd.concat(dfs, ignore_index=True)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album,song name
0,0.305,0.872,2,-7.313,1,0.0503,0.007860,0.800000,0.2610,0.648,167.959,audio_features,7c378mlmubSu7NGkLFa4sN,spotify:track:7c378mlmubSu7NGkLFa4sN,https://api.spotify.com/v1/tracks/7c378mlmubSu...,https://api.spotify.com/v1/audio-analysis/7c37...,287880,4,OK Computer,Airbag
1,0.251,0.848,5,-6.501,1,0.0579,0.037700,0.005910,0.0545,0.207,163.709,audio_features,6LgJvl0Xdtc73RJ1mmpotq,spotify:track:6LgJvl0Xdtc73RJ1mmpotq,https://api.spotify.com/v1/tracks/6LgJvl0Xdtc7...,https://api.spotify.com/v1/audio-analysis/6LgJ...,387213,4,OK Computer,Paranoid Android
2,0.312,0.595,0,-8.919,1,0.0298,0.048400,0.001390,0.1030,0.337,150.502,audio_features,2CVV8PtUYYsux8XOzWkCP0,spotify:track:2CVV8PtUYYsux8XOzWkCP0,https://api.spotify.com/v1/tracks/2CVV8PtUYYsu...,https://api.spotify.com/v1/audio-analysis/2CVV...,267693,3,OK Computer,Subterranean Homesick Alien
3,0.293,0.276,4,-11.357,0,0.0349,0.224000,0.122000,0.1670,0.195,121.603,audio_features,0z1o5L7HJx562xZSATcIpY,spotify:track:0z1o5L7HJx562xZSATcIpY,https://api.spotify.com/v1/tracks/0z1o5L7HJx56...,https://api.spotify.com/v1/audio-analysis/0z1o...,267187,4,OK Computer,Exit Music (For A Film)
4,0.351,0.676,9,-9.017,1,0.0313,0.000121,0.113000,0.1800,0.143,102.460,audio_features,2fuYa3Lx06QQJAm0MjztKr,spotify:track:2fuYa3Lx06QQJAm0MjztKr,https://api.spotify.com/v1/tracks/2fuYa3Lx06QQ...,https://api.spotify.com/v1/audio-analysis/2fuY...,299560,4,OK Computer,Let Down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,0.445,0.680,8,-11.338,1,0.0528,0.378000,0.000209,0.1020,0.750,163.664,audio_features,5Bh8l8evdBSIoaK6EP1bWI,spotify:track:5Bh8l8evdBSIoaK6EP1bWI,https://api.spotify.com/v1/tracks/5Bh8l8evdBSI...,https://api.spotify.com/v1/audio-analysis/5Bh8...,195760,4,Bridge Over Troubled Water,Baby Driver
5800,0.411,0.381,11,-12.361,1,0.0286,0.130000,0.257000,0.1320,0.524,76.963,audio_features,5MbXzXGbqobR8xPVPs8OXA,spotify:track:5MbXzXGbqobR8xPVPs8OXA,https://api.spotify.com/v1/tracks/5MbXzXGbqobR...,https://api.spotify.com/v1/audio-analysis/5MbX...,238413,4,Bridge Over Troubled Water,The Only Living Boy in New York
5801,0.745,0.771,8,-10.067,1,0.0352,0.355000,0.000315,0.0791,0.932,117.209,audio_features,0nNI8eyMjDGjPDuQq87aJb,spotify:track:0nNI8eyMjDGjPDuQq87aJb,https://api.spotify.com/v1/tracks/0nNI8eyMjDGj...,https://api.spotify.com/v1/audio-analysis/0nNI...,166173,4,Bridge Over Troubled Water,Why Don't You Write Me
5802,0.431,0.963,6,-7.823,1,0.0677,0.177000,0.000071,0.8330,0.637,152.527,audio_features,1tk4uuDWxk0DTx2X5rGSxe,spotify:track:1tk4uuDWxk0DTx2X5rGSxe,https://api.spotify.com/v1/tracks/1tk4uuDWxk0D...,https://api.spotify.com/v1/audio-analysis/1tk4...,172773,4,Bridge Over Troubled Water,"Bye Bye Love - Live at Memorial Auditorium, Bu..."


In [15]:
get_audio_features(second_top_500_names)

John Lennon / Plastic Ono Band
Run the Jewels 3
Just for a Day
What's Your Pleasure?
GREY Area
The Mantle
Benji
Promises
Angel Dust
High Violet
Imperial
Dirty
Zen Arcade
Alfredo
Antics
No se encontró ningún álbum con el nombre 'Jenny Death: The Powers That B Disc 2'.
Pure Heroine
Three Imaginary Boys
An Evening With Silk Sonic
The Infamous
...I Care Because You Do
Around the Fur
You Forgot It in People
Zuu
Viva la vida or Death and All His Friends
Disraeli Gears
The Earth Is Not a Cold Dead Place
Faith
Congratulations
Load
You're Dead!
Different Class
Weezer [Green Album]
Tommy
James Blake
2
Double Nickels on the Dime
Michigan
Death Magnetic
The Great Annihilator
Humanz
Ginger
You Will Never Know Why
Mount Eerie
Man on the Moon: The End of Day
Q: Are We Not Men? A: We Are Devo!
Crystal Castles
In absentia
Silent Alarm
Crack the Skye
Aja
The Ape of Naples
LP1
Children of God
Highway to Hell
Human
SAWAYAMA
I Could Live in Hope
Visions of Bodies Being Burned
No se encontró ningún álbum co

Head Over Heels
Play
Encore
Reckoning
Yank Crime
The Black Album
Bad Moon Rising
Show No Mercy
Brothers in Arms
Exuma
Relayer
Document
Gish
No se encontraron caracteristicas de audio para las canciones de Ocean Rain
Yeti
A Farewell to Kings
The Boatman's Call
British Steel
Chutes Too Narrow
Volta
TNT
Daughters
Painful
Wolfgang Amadeus Phoenix
Ghosts of the Great Highway
Vitalogy
Los Angeles
Kaputt
No se encontraron caracteristicas de audio para las canciones de Iowa
Eversince
Combat Rock
Dirty Computer
Testing
Pure Comedy
The Magnolia Electric Co.
あくまのうた (Akuma no uta)
Deliverance
Sad Wings of Destiny
Can't Buy a Thrill
Oh, Inverted World
The Boy With the Arab Strap
Brave New World
Roman Candle
Use Your Illusion I
Southernplayalisticadillacmuzik
Urban Hymns
The Soft Parade
Perdition City
Trouble Will Find Me
Expensive Shit


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album,song name
0,0.305,0.872,2,-7.313,1,0.0503,0.007860,0.80000,0.2610,0.648,167.959,audio_features,7c378mlmubSu7NGkLFa4sN,spotify:track:7c378mlmubSu7NGkLFa4sN,https://api.spotify.com/v1/tracks/7c378mlmubSu...,https://api.spotify.com/v1/audio-analysis/7c37...,287880,4,OK Computer,Airbag
1,0.251,0.848,5,-6.501,1,0.0579,0.037700,0.00591,0.0545,0.207,163.709,audio_features,6LgJvl0Xdtc73RJ1mmpotq,spotify:track:6LgJvl0Xdtc73RJ1mmpotq,https://api.spotify.com/v1/tracks/6LgJvl0Xdtc7...,https://api.spotify.com/v1/audio-analysis/6LgJ...,387213,4,OK Computer,Paranoid Android
2,0.312,0.595,0,-8.919,1,0.0298,0.048400,0.00139,0.1030,0.337,150.502,audio_features,2CVV8PtUYYsux8XOzWkCP0,spotify:track:2CVV8PtUYYsux8XOzWkCP0,https://api.spotify.com/v1/tracks/2CVV8PtUYYsu...,https://api.spotify.com/v1/audio-analysis/2CVV...,267693,3,OK Computer,Subterranean Homesick Alien
3,0.293,0.276,4,-11.357,0,0.0349,0.224000,0.12200,0.1670,0.195,121.603,audio_features,0z1o5L7HJx562xZSATcIpY,spotify:track:0z1o5L7HJx562xZSATcIpY,https://api.spotify.com/v1/tracks/0z1o5L7HJx56...,https://api.spotify.com/v1/audio-analysis/0z1o...,267187,4,OK Computer,Exit Music (For A Film)
4,0.351,0.676,9,-9.017,1,0.0313,0.000121,0.11300,0.1800,0.143,102.460,audio_features,2fuYa3Lx06QQJAm0MjztKr,spotify:track:2fuYa3Lx06QQJAm0MjztKr,https://api.spotify.com/v1/tracks/2fuYa3Lx06QQ...,https://api.spotify.com/v1/audio-analysis/2fuY...,299560,4,OK Computer,Let Down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11154,0.470,0.700,5,-8.600,0,0.0425,0.782000,0.24400,0.2550,0.446,167.749,audio_features,7mxVCiILTA07FmLly7xvpm,spotify:track:7mxVCiILTA07FmLly7xvpm,https://api.spotify.com/v1/tracks/7mxVCiILTA07...,https://api.spotify.com/v1/audio-analysis/7mxV...,300733,4,Trouble Will Find Me,Humiliation
11155,0.502,0.656,9,-7.340,1,0.0435,0.799000,0.10900,0.0895,0.275,110.157,audio_features,1oNDnnBojo1hkFATFEp1CM,spotify:track:1oNDnnBojo1hkFATFEp1CM,https://api.spotify.com/v1/tracks/1oNDnnBojo1h...,https://api.spotify.com/v1/audio-analysis/1oND...,276200,4,Trouble Will Find Me,Pink Rabbits
11156,0.488,0.514,8,-12.021,1,0.0309,0.794000,0.00508,0.0979,0.104,114.079,audio_features,4hizMtcebzlqsZqp1Y6yJo,spotify:track:4hizMtcebzlqsZqp1Y6yJo,https://api.spotify.com/v1/tracks/4hizMtcebzlq...,https://api.spotify.com/v1/audio-analysis/4hiz...,253454,5,Trouble Will Find Me,Hard to Find
11157,0.612,0.876,5,-7.668,0,0.1100,0.220000,0.39900,0.0863,0.677,123.866,audio_features,2yBjFF19OmukrC7deP7LfZ,spotify:track:2yBjFF19OmukrC7deP7LfZ,https://api.spotify.com/v1/tracks/2yBjFF19Omuk...,https://api.spotify.com/v1/audio-analysis/2yBj...,793200,4,Expensive Shit,Expensive Shit


In [16]:
data = pd.concat(dfs, ignore_index=True)
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,album,song name
0,0.305,0.872,2,-7.313,1,0.0503,0.007860,0.80000,0.2610,0.648,167.959,audio_features,7c378mlmubSu7NGkLFa4sN,spotify:track:7c378mlmubSu7NGkLFa4sN,https://api.spotify.com/v1/tracks/7c378mlmubSu...,https://api.spotify.com/v1/audio-analysis/7c37...,287880,4,OK Computer,Airbag
1,0.251,0.848,5,-6.501,1,0.0579,0.037700,0.00591,0.0545,0.207,163.709,audio_features,6LgJvl0Xdtc73RJ1mmpotq,spotify:track:6LgJvl0Xdtc73RJ1mmpotq,https://api.spotify.com/v1/tracks/6LgJvl0Xdtc7...,https://api.spotify.com/v1/audio-analysis/6LgJ...,387213,4,OK Computer,Paranoid Android
2,0.312,0.595,0,-8.919,1,0.0298,0.048400,0.00139,0.1030,0.337,150.502,audio_features,2CVV8PtUYYsux8XOzWkCP0,spotify:track:2CVV8PtUYYsux8XOzWkCP0,https://api.spotify.com/v1/tracks/2CVV8PtUYYsu...,https://api.spotify.com/v1/audio-analysis/2CVV...,267693,3,OK Computer,Subterranean Homesick Alien
3,0.293,0.276,4,-11.357,0,0.0349,0.224000,0.12200,0.1670,0.195,121.603,audio_features,0z1o5L7HJx562xZSATcIpY,spotify:track:0z1o5L7HJx562xZSATcIpY,https://api.spotify.com/v1/tracks/0z1o5L7HJx56...,https://api.spotify.com/v1/audio-analysis/0z1o...,267187,4,OK Computer,Exit Music (For A Film)
4,0.351,0.676,9,-9.017,1,0.0313,0.000121,0.11300,0.1800,0.143,102.460,audio_features,2fuYa3Lx06QQJAm0MjztKr,spotify:track:2fuYa3Lx06QQJAm0MjztKr,https://api.spotify.com/v1/tracks/2fuYa3Lx06QQ...,https://api.spotify.com/v1/audio-analysis/2fuY...,299560,4,OK Computer,Let Down
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11154,0.470,0.700,5,-8.600,0,0.0425,0.782000,0.24400,0.2550,0.446,167.749,audio_features,7mxVCiILTA07FmLly7xvpm,spotify:track:7mxVCiILTA07FmLly7xvpm,https://api.spotify.com/v1/tracks/7mxVCiILTA07...,https://api.spotify.com/v1/audio-analysis/7mxV...,300733,4,Trouble Will Find Me,Humiliation
11155,0.502,0.656,9,-7.340,1,0.0435,0.799000,0.10900,0.0895,0.275,110.157,audio_features,1oNDnnBojo1hkFATFEp1CM,spotify:track:1oNDnnBojo1hkFATFEp1CM,https://api.spotify.com/v1/tracks/1oNDnnBojo1h...,https://api.spotify.com/v1/audio-analysis/1oND...,276200,4,Trouble Will Find Me,Pink Rabbits
11156,0.488,0.514,8,-12.021,1,0.0309,0.794000,0.00508,0.0979,0.104,114.079,audio_features,4hizMtcebzlqsZqp1Y6yJo,spotify:track:4hizMtcebzlqsZqp1Y6yJo,https://api.spotify.com/v1/tracks/4hizMtcebzlq...,https://api.spotify.com/v1/audio-analysis/4hiz...,253454,5,Trouble Will Find Me,Hard to Find
11157,0.612,0.876,5,-7.668,0,0.1100,0.220000,0.39900,0.0863,0.677,123.866,audio_features,2yBjFF19OmukrC7deP7LfZ,spotify:track:2yBjFF19OmukrC7deP7LfZ,https://api.spotify.com/v1/tracks/2yBjFF19Omuk...,https://api.spotify.com/v1/audio-analysis/2yBj...,793200,4,Expensive Shit,Expensive Shit


Here i export the data to use it in another notebook

In [17]:
data.to_csv("first_1000_albums_features")